Latihan Peramalam Menggunakan Data Cuaca dari openWeather API
Pendeklarasian
# pip install xgboost

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [2]:
# fungsi untuk mengambil data cuaca dari open weather API
def get_weather_data(api_key, city):
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city,"appid": api_key,"units": "metric"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

In [5]:
# fungsi untuk mengubah ke data cuaca ke dataframe
def parse_weather_data(weather_data):
    df = pd.json_normalize(weather_data, record_path=['list'])
    df['dt'] = pd.to_datetime(df['dt'], unit='s')
    df.set_index('dt', inplace=True)
    df = df[['main.temp_max']] # ambil suhu maksimum
    df.columns = ['Temp_Max'] #ubah nama kolom
    return df
# menggunakan API key dan nama kota
API_KEY = '68433059aae9bc07f30ef23c973276a5' #ganti dengan API key anda
CITY = 'Jakarta'

mengambil data cuaca lalu memparsing ke dataframe

In [6]:
#Ambil data cuaca
weather_data = get_weather_data(API_KEY, CITY)
#parse data ke dataframe
data = parse_weather_data(weather_data)
#siapkan data untuk model
data['Hour'] = data.index.hour
data['Day'] = data.index.dayofweek
data['Month'] = data.index.month
#memilih fitur dan target
X = data[['Hour', 'Day', 'Month']]
y = data['Temp_Max']


In [7]:
# membagi dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#membuat DMatrix untuk XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
#set parameter untuk XGBoost
params = {'objective': 'reg.squarederror', 'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}